<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#User-and-Item-Based-Filtering" data-toc-modified-id="User-and-Item-Based-Filtering-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>User and Item Based Filtering</a></span><ul class="toc-item"><li><span><a href="#Data" data-toc-modified-id="Data-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Data</a></span></li></ul></li><li><span><a href="#User-based-Filtering" data-toc-modified-id="User-based-Filtering-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>User-based Filtering</a></span><ul class="toc-item"><li><span><a href="#Step-1:-Identify-similar-users" data-toc-modified-id="Step-1:-Identify-similar-users-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Step 1: Identify similar users</a></span></li><li><span><a href="#Ranking-the-critics" data-toc-modified-id="Ranking-the-critics-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Ranking the critics</a></span></li><li><span><a href="#Step-2:-Recommending-Items" data-toc-modified-id="Step-2:-Recommending-Items-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Step 2: Recommending Items</a></span></li><li><span><a href="#Item-based-Filtering" data-toc-modified-id="Item-based-Filtering-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Item-based Filtering</a></span></li></ul></li><li><span><a href="#User-or-Item-based-Filtering?" data-toc-modified-id="User-or-Item-based-Filtering?-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>User or Item-based Filtering?</a></span></li></ul></div>

# User and Item Based Filtering
> Two different ways to do collaborative filtering

- toc: true 
- badges: true
- comments: true
- search_exclude: false
- categories: [machinelearning]

Collaborative Filtering is a method in recommendation systems that searches for similar objects in order to create a ranked list of suggestions. In general, we distinguish between two different approaches:

- User-based filtering: we first identify the most similar users to the one for whom we want to give recommendations.
- Item-based filtering: we first search for the most similar items to the ones that are most relevant to the user (e.g., most liked by the user)

We will look at both approaches and discuss their advantages. This blog post is very much inspired by Chapter 2 of Toby Segaran's classic book "Programming Collective Intelligence" (O'Reilly, 2007).

## Data

For our discussion we need some recommendation data. There are several ways of expressing a user's preference, such as binary distinctions (booked a hotel vs. didn't book the hotel) or more explicit feedback in the form of graded reviews.

We take the exemplary data below (taken from Segaran's book) and compute the recommendations using the Pandas library. 

In [1]:
critics={'Lisa Rose': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.5,
      'Just My Luck': 3.0, 'Superman Returns': 3.5, 'You, Me and Dupree': 2.5,
      'The Night Listener': 3.0},
     'Gene Seymour': {'Lady in the Water': 3.0, 'Snakes on a Plane': 3.5,
      'Just My Luck': 1.5, 'Superman Returns': 5.0, 'The Night Listener': 3.0,
      'You, Me and Dupree': 3.5},
     'Michael Phillips': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.0,
      'Superman Returns': 3.5, 'The Night Listener': 4.0},
     'Claudia Puig': {'Snakes on a Plane': 3.5, 'Just My Luck': 3.0,
      'The Night Listener': 4.5, 'Superman Returns': 4.0,
      'You, Me and Dupree': 2.5},
     'Mick LaSalle': {'Lady in the Water': 3.0, 'Snakes on a Plane': 4.0,
      'Just My Luck': 2.0, 'Superman Returns': 3.0, 'The Night Listener': 3.0,
      'You, Me and Dupree': 2.0},
     'Jack Matthews': {'Lady in the Water': 3.0, 'Snakes on a Plane': 4.0,
      'The Night Listener': 3.0, 'Superman Returns': 5.0, 'You, Me and Dupree': 3.5},
     'Toby': {'Snakes on a Plane':4.5,'You, Me and Dupree':1.0,'Superman Returns':4.0}
}

In [2]:
import pandas as pd
critics = pd.DataFrame(critics).T

In [3]:
critics

,Lady in the Water,Snakes on a Plane,Just My Luck,Superman Returns,"You, Me and Dupree",The Night Listener
Lisa Rose,2.5,3.5,3.0,3.5,2.5,3.0
Gene Seymour,3.0,3.5,1.5,5.0,3.5,3.0
Michael Phillips,2.5,3.0,NaN,3.5,NaN,4.0
Claudia Puig,NaN,3.5,3.0,4.0,2.5,4.5
Mick LaSalle,3.0,4.0,2.0,3.0,2.0,3.0
Jack Matthews,3.0,4.0,NaN,5.0,3.5,3.0
Toby,NaN,4.5,NaN,4.0,1.0,NaN


The data frame shows our toy data set as an input for recommendations. Usually, the matrix is much bigger and way more sparse. Yet, it illustrates the fact that some users (such as Toby) haven't rated (and thus might not have seen) all the movies. We would like to get a ranking of the missing movies so that we can recommend the movie which most likely fits to their preferences.

In the collaborative filtering approach (be it user or item based), there are the two main steps to get to this ranking (here illustrated with the user-based approach for our movie toy data set):

1. Define a similarity between objects (either users or items). This step is different from user and item based recommenders and gives them their respective names. In the user-based approach, the similarity is defined in terms of how the users rated the movies. If users give the same or a very similar rating to the same movie, they are considered more similar than if they give very different ratings. We will see below how we can quantify this to arrive at a similarity matrix between users.
2. The second step takes the previously defined similarity metric and uses it to compute a weighted average to obtain the final recommendation scores. Once we identified a similarity among users, we will check all the missing movies and weigh the ratings that the other users gave depending on their similarity to the user at hand. 

# User-based Filtering

Now we want to go through the different steps for user-based filtering and compute the similarities and rankings. 

## Step 1: Identify similar users

There are multiple ways to define the similarity among users based on the ratings they gave to the movies. However, there is one particular method that a) is easy to compute with Pandas data frames and b) works particularly well when the ratings are not normalized: the **Pearson correlation** coefficient. 

Pearson correlation has values between -1 and +1, where a negative value indicates that the ratings from two users tend to correlate negatively whereas a positive value implies positive correlation. Negative correlation means that a very high rating from the first user would imply a rather low rating from the other while a positive correlation means that both users's ratings are similar in their magnitude.

In Pandas, there is a very convenient way to get the Pearson correlation coefficients for a data frame by using the `corr()` method. You just have to decide for which items in our data frame the correlation should be computed, for the users or the movies. In our case, we're interested in the users, that's why we have to transpose the data frame first (by using the `.T` shortcut, i.e., `critics.T`) before applying the correlation method.

In [4]:
critics.T.corr(method='pearson')

,Lisa Rose,Gene Seymour,Michael Phillips,Claudia Puig,Mick LaSalle,Jack Matthews,Toby
Lisa Rose,1.000000,0.396059,0.404520,0.566947,0.594089,0.747018,0.991241
Gene Seymour,0.396059,1.000000,0.204598,0.314970,0.411765,0.963796,0.381246
Michael Phillips,0.404520,0.204598,1.000000,1.000000,-0.258199,0.134840,-1.000000
Claudia Puig,0.566947,0.314970,1.000000,1.000000,0.566947,0.028571,0.893405
Mick LaSalle,0.594089,0.411765,-0.258199,0.566947,1.000000,0.211289,0.924473
Jack Matthews,0.747018,0.963796,0.134840,0.028571,0.211289,1.000000,0.662849
Toby,0.991241,0.381246,-1.000000,0.893405,0.924473,0.662849,1.000000


As can be seen in the correlation matrix, some users are more similar to each other than others. For example, users Toby and Michael Phillips show completely opposite behavior, indicated by a Pearson coefficient of -1. This is because they only share ratings for two movies ('Snakes on a Plane' and 'Superman Returns') and those ratings are completely negatively correlated. Compared to their respective average values (3.25 for Michael Phillips and 4.25 for Toby), they gave opposite directions for these two movies. While Phillips gave a lower rating for 'Snakes' (3.0), Toby gave a higher value (4.5) and vice versa for 'Superman Returns' (4.0 in Toby's case and 3.5 in Phillips').

On the other hand, Toby and Lisa Rose's ratings are very similar to each other, showing a Pearson value of 0.99.

Users always have a Pearson correlation coefficient of 1 to themselves because, obviously, their ratings show a perfect correlation. The matrix is also symmetrical as the correlation is not dependent on order. 

In [5]:
import numpy as np

def euclidean(a, b):
  eucl = np.square(a-b).sum()
  return 1/(1+eucl)

In [6]:
critics.T.corr(method=euclidean)

,Lisa Rose,Gene Seymour,Michael Phillips,Claudia Puig,Mick LaSalle,Jack Matthews,Toby
Lisa Rose,1.000000,0.148148,0.444444,0.285714,0.333333,0.210526,0.222222
Gene Seymour,0.148148,1.000000,0.210526,0.133333,0.129032,0.800000,0.108108
Michael Phillips,0.444444,0.210526,1.000000,0.571429,0.285714,0.181818,0.285714
Claudia Puig,0.285714,0.133333,0.571429,1.000000,0.173913,0.181818,0.235294
Mick LaSalle,0.333333,0.129032,0.285714,0.173913,1.000000,0.137931,0.307692
Jack Matthews,0.210526,0.800000,0.181818,0.181818,0.137931,1.000000,0.117647
Toby,0.222222,0.108108,0.285714,0.235294,0.307692,0.117647,1.000000


## Ranking the critics

With the Pearson correlation matrix we can compute a ranking of critics for each user. To this end, we define the function `top_matches` that makes use of the `corr()` method of data frames for the similary scores. To get a proper similarity ranking, we first identify all users other than the one under investigation and return a Pandas Series that is created by indexing the correlation matrix for the respective user. Finally, the series is sorted in reverse order to return the top similar users.

In [7]:
def top_matches(critics, person, n=10, method='pearson'):
  other_people = [p for p in critics.T.columns if p != person]
  corr_persons = critics.T.corr(method=method)
  scores = corr_persons.loc[other_people, person]
  scores.sort_values(ascending=False, inplace=True)
  return scores.head(n)


As mentioned above, the most similar user to Toby is Lisa Rose, showing a correlation value of almost 1, whereas the least similar is Michael Phillips.

In [8]:
top_matches(critics, 'Toby')

Lisa Rose           0.991241
Mick LaSalle        0.924473
Claudia Puig        0.893405
Jack Matthews       0.662849
Gene Seymour        0.381246
Michael Phillips   -1.000000
Name: Toby, dtype: float64

## Step 2: Recommending Items

Having defined the similarity of users, we can move to the second step: how to compute the actual recommendations for user Toby. For this, we first select those movies in our toy dataset that Toby hasn't ranked yet (assuming he hasn't watched them either). These missing movies are the basis for our recommendations. All we need to do now is rank them in the appropriate order so that the movie that we assume he would rate highest comes up first in our ranking.

This begs the question how we make the predictions. The basic idea is pretty simple: we take the ratings from other users for the missing movies and compute the weighted average of the rankings for each movie. But where do we take the weights from? It's very easy. We take advantage of the similarities that we worked out above. Thus, the final score for each movie (for user Toby) is calculated by taking the sum of the rating times the similarity (to Toby) for each movie divided by the overall sum of the similarities.

$ score_{movie} = \frac{\sum_{i=1}^{users} sim_{user_i, Toby} * rating_{user_i, movie}}{\sum_{i=1}^{users} sim_{user_i, Toby}}$

In Pandas, we can use the `isnull()` method on the Series to identify all the missing movies for the user. Given the similarity rankings from our previously defined function `top_matches`, we remove those users whose correlation coefficient is negative. The rationale is that we only want to stick to users that rate positively correlated with Toby. We then index into the ratings data frame, extracting only those cells with similar users and missing films. For the actual computation of the scores we need two objects: 

1. the subset of the cells from the original data frame of ratings giving the similar users and the missing movies (`cores`)
2. the filtered user similarities where all negatively correlated users to Toby have been discarded (`sims`)

In [9]:
def get_recommendations(critics, person):
  missing_films = critics.T[critics.T[person].isnull()].index.values
  person_similarities = top_matches(critics, person, n=len(critics))
  person_similarities = person_similarities[person_similarities > 0]
  movies_scores = critics.loc[person_similarities.index, missing_films]
  return movies_scores, person_similarities

In [10]:
scores, sims = get_recommendations(critics, 'Toby')
scores

,Lady in the Water,Just My Luck,The Night Listener
Lisa Rose,2.5,3.0,3.0
Mick LaSalle,3.0,2.0,3.0
Claudia Puig,NaN,3.0,4.5
Jack Matthews,3.0,NaN,3.0
Gene Seymour,3.0,1.5,3.0


In [11]:
sims

Lisa Rose        0.991241
Mick LaSalle     0.924473
Claudia Puig     0.893405
Jack Matthews    0.662849
Gene Seymour     0.381246
Name: Toby, dtype: float64

With these two objects we can first compute the numerator of the final score by taking the ratings for the missing movies and multiply them by the respective user's similarity to Toby.

In [12]:
films_overview = scores.T * sims
films_overview

,Lisa Rose,Mick LaSalle,Claudia Puig,Jack Matthews,Gene Seymour
Lady in the Water,2.478102,2.773420,NaN,1.988547,1.143739
Just My Luck,2.973722,1.848947,2.680215,NaN,0.571870
The Night Listener,2.973722,2.773420,4.020323,1.988547,1.143739


After that, we can easily get the sum of these weighted scores for each movie.

In [13]:
films_overview.sum(axis=1)

Lady in the Water      8.383808
Just My Luck           8.074754
The Night Listener    12.899752
dtype: float64

All that is left now is to compute the denominator. For this, we take the non-null ratings as a mask for the similarities. The reason for this step is to remove individual similarities from the totals if the user didn't rate the respective movie. Pandas's weak typing comes in handy at this step. The boolean return values of the `notnull()` method are turned into integers (0 for `False`), which nullify the factor for the respective user. At the end, we obtain a similarity total for each movie.

In [14]:
totals = (scores.notnull().T * sims).sum(axis=1)
totals

Lady in the Water     2.959810
Just My Luck          3.190366
The Night Listener    3.853215
dtype: float64

Finally, we can compute the actual predicted movie ratings for user Toby by taking the sums of the weighted scores and dividing them by the totals. In our toy example, the movie 'The Night Listener' gets the highest rating with 3.35 and would thus be our first choice for a recommendation to Toby.

In [15]:
recs = films_overview.sum(axis=1) / totals
recs.sort_values()

Just My Luck          2.530981
Lady in the Water     2.832550
The Night Listener    3.347790
dtype: float64

## Item-based Filtering

So far, we've looked at the user similarities first to reduce the search space for our recommendations. However, this comes with a downside when your matrix of ratings includes a large number of users and shows a lot of sparsity, thus making it hard to compute similarities among users when there is little to no overlap.

An alternative approach, called item-based filtering, tries to remedy these problems by dispensing with the user similarity step and searching for similar items right away (hence the name). With our toy dataframe this is very easy as we only need to apply the `corr()` method again, this time on the items and not on the users:

In [16]:
critics.corr(method=euclidean)

,Lady in the Water,Snakes on a Plane,Just My Luck,Superman Returns,"You, Me and Dupree",The Night Listener
Lady in the Water,1.000000,0.222222,0.222222,0.090909,0.400000,0.285714
Snakes on a Plane,0.222222,1.000000,0.105263,0.166667,0.051282,0.181818
Just My Luck,0.222222,0.105263,1.000000,0.064516,0.181818,0.153846
Superman Returns,0.090909,0.166667,0.064516,1.000000,0.053333,0.102564
"You, Me and Dupree",0.400000,0.051282,0.181818,0.053333,1.000000,0.148148
The Night Listener,0.285714,0.181818,0.153846,0.102564,0.148148,1.000000


Again, the same principles apply that we mentioned above for the user similarities. A score close to 1 indicates that the movies are very similar. For example, the movies 'Snakes on a Plane' and 'Lady in the Water' seem to be alike based on the users's ratings whereas 'Just my Luck' and 'Lady in the Water' are quite dissimilar. 

To compute the final recommendation scores all we need to do is to identify the missing movies of a given user (Toby) and compute a weighted average of their similarity to all movies the user has already rated together with their similarity to the missing movies.

In [17]:
def get_item_recommendations(critics, person, method='pearson'):
  missing_films = critics.T[critics.T[person].isnull()].index.values
  movie_similarities = critics.corr(method=method)
  movie_similarities = movie_similarities.loc[~movie_similarities.index.isin(missing_films), missing_films]
  person_ratings = critics.loc[person, ~critics.columns.isin(missing_films)]
  
  return movie_similarities, person_ratings

In [18]:
ms, pr = get_item_recommendations(critics, 'Toby', method=euclidean)
ms

,Lady in the Water,Just My Luck,The Night Listener
Snakes on a Plane,0.222222,0.105263,0.181818
Superman Returns,0.090909,0.064516,0.102564
"You, Me and Dupree",0.400000,0.181818,0.148148


In [19]:
pr

Snakes on a Plane     4.5
Superman Returns      4.0
You, Me and Dupree    1.0
Name: Toby, dtype: float64

Like above, we now take the missing movies and weigh them with the similarities that we defined above. The result is an item-based recommendation score that indicates which movies we should recommend next to the user (in this case Toby).

In [20]:
(ms.T * pr).sum(axis=1)

Lady in the Water     1.763636
Just My Luck          0.913567
The Night Listener    1.376586
dtype: float64

In [21]:
item_recos = (ms.T*pr).sum(axis=1) / ms.sum()
item_recos

Lady in the Water     2.473088
Just My Luck          2.598332
The Night Listener    3.182635
dtype: float64

# User or Item-based Filtering?

As shown above, user and item-based filtering mainly differ in the first step: we either have to compute similarities for users or times first. With a very large database, item-based filtering is to be preferred as it is significantly faster to compare the items for users to identify the similarities than to compare a given user to all other users